In [32]:
import required_funcs as rf
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from torch.utils.data import Dataset
import torch
import pandas as pd

from sklearn.metrics import precision_recall_fscore_support, accuracy_score

In [33]:
model_name = (
    "xlm-roberta-base"  # or any other pre-trained MLM model available on Hugging Face
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/ravimaith/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sen

In [34]:
num_labels = 2  # For binary classification
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=num_labels
)

loading configuration file config.json from cache at /Users/ravimaith/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading weights file pytorch_model.bin from cache at /Users/ravimaith/.cache/huggingface/hub/models--xlm-robe

In [35]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text, return_tensors="pt", padding="max_length", truncation=True
        )
        value = {key: torch.tensor(val[0]) for key, val in encoding.items()}
        value["label"] = torch.tensor(label)
        return value

In [36]:
train = pd.read_json("dataset/training/EXIST2023_training.json", orient="index")
train = train.sample(frac=1).reset_index(drop=True)

In [37]:
# loading the validation datset
validation = pd.read_json("dataset/dev/EXIST2023_dev.json", orient="index")
validation = validation.sample(frac=1).reset_index(drop=True)

In [38]:
train_labels1 = train["labels_task1"]
train_lang = train["lang"]
train_text = train["tweet"]

train_labels_pre = rf.majority_vote(train_labels1)
train_labels = rf.label_convertor(train_labels_pre)

In [39]:
valid_labels1 = validation["labels_task1"]
valid_lang = validation["lang"]
valid_text = validation["tweet"]

valid_labels_pre = rf.majority_vote(valid_labels1)
valid_labels = rf.label_convertor(valid_labels_pre)

In [40]:
train_dataset = CustomDataset(train_text, train_labels, tokenizer)
valid_dataset = CustomDataset(valid_text, valid_labels, tokenizer)

In [41]:
training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
)

using `logging_steps` to initialize `eval_steps` to 10
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [42]:
train_dataset.__getitem__(0)

/var/folders/fv/2rfdp87n0299b4fs27y2y18c0000gn/T/ipykernel_18463/2044514904.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = {key: torch.tensor(val[0]) for key, val in encoding.items()}


{'input_ids': tensor([     0,   1374,    284,   8876,  15277,   1255,    594,  96420,      5,
             27,  21303, 109578, 240116,  56611,  41329,    370, 235817,     11,
             27,   3757,    493,    220,  44543, 148125,     11,  33007,     78,
              5,     27,   1788,     41,  21303,    120,     22,  86801,     27,
            365, 167264,      7,   3757,  11007,    120,  34387,  21303,  33587,
              5,      2,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1

In [44]:
torch.device("mps")

device(type='mps')

In [45]:
trainer.train()

***** Running training *****
  Num examples = 6920
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2595


  0%|          | 0/2595 [00:00<?, ?it/s]

/var/folders/fv/2rfdp87n0299b4fs27y2y18c0000gn/T/ipykernel_18463/2044514904.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  value = {key: torch.tensor(val[0]) for key, val in encoding.items()}


KeyboardInterrupt: 